<a href="https://colab.research.google.com/github/Auroraleone/MLDLproject-/blob/main/Ensemple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##ENSEMBLE##


In [ ]:
## MIOU.py ##

def calculate_iou(true_int, pred_int):
    lower_true = true_int[0]
    upper_true = true_int[1]
    lower_pred = pred_int[0]
    upper_pred = pred_int[1]

    if upper_true < lower_pred or lower_true > upper_pred:
        return 0.0
    else:
        intersection = max(0, min(upper_true, upper_pred) - max(lower_true, lower_pred))
        union = max(upper_true, upper_pred) - min(lower_true, lower_pred)
        return intersection / union


def calculate_miou(true_int, pred_ints):
    ious = [calculate_iou(true_int, pred_int) for pred_int in pred_ints]
    return sum(ious) / len(pred_ints)


In [ ]:
## update_json_with_intervals.py ##

import json


def update_json_with_intervals(json_file_path, intervals_list):
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    if len(data['results']) < len(intervals_list):
        print("The numer of lists does not match the results in the JSON file")
        return

    for i in range(min(len(data['results']), len(intervals_list))):
        if len(intervals_list[i]) != 5:
            print(f"List of intervals in position {i} does not have length 5.")
            return
        data['results'][i]['predicted_times'] = intervals_list[i]

    with open(json_file_path, 'w') as file:
        json.dump(data, file, indent=4)
    print(f"File JSON updated and saved in {json_file_path}")


In [ ]:
## val_to_csv.py ##

import json
import pandas as pd


def read_file_json(file_name):
    with open(file_name, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data


def create_list_of_dictionaries(data):
    list_of_dictionaries = []
    for key, value in data.items():
        dictionary = {
            'clip_uid': key,
            'fps': value['fps'],
            'num_frames': value['num_frames'],
            'timestamps': value['timestamps'],
            'exact_times': value['exact_times'],
            'sentences': value['sentences'],
            'annotation_uids': value['annotation_uids'],
            'query_idx': value['query_idx']
        }
        list_of_dictionaries.append(dictionary)
    return list_of_dictionaries


def from_dict_to_df(dict):
    df = pd.DataFrame({
        'clip_uid': [dict['clip_uid']] * len(dict['timestamps']),
        'fps': [dict['fps']] * len(dict['timestamps']),
        'num_frames': [dict['num_frames']] * len(dict['timestamps']),
        'timestamp': dict['timestamps'],
        'exact_time': dict['exact_times'],
        'sentences': dict['sentences'],
        'annotation_uids': dict['annotation_uids'],
        'query_idx': dict['query_idx']
    })
    return df

json_file_name = 'val.json'
data = read_file_json(json_file_name)
list_of_dictionaries = create_list_of_dictionaries(data)

pd.set_option('display.max_columns', None)

df_list = []
for i in range(len(list_of_dictionaries)):
    df_list.append(from_dict_to_df(list_of_dictionaries[i]))

df_concatenated = pd.concat(df_list, ignore_index=True)

print(df_concatenated)

file_name = 'VALIDATION.csv'
df_concatenated.to_csv(file_name, index=False)

print(f"DataFrame salved in '{file_name}'.")


In [ ]:
## vsl_pred_to_csv.py ##

def json_to_csv(file_path):

    import pandas as pd
    import os

    df = pd.read_json(file_path)

    df['clip_uid'] = df['results'].apply(lambda x: x['clip_uid'])
    df['annotation_uid'] = df['results'].apply(lambda x: x['annotation_uid'])
    df['query_idx'] = df['results'].apply(lambda x: x['query_idx'])

    predicted_times = df['results'].apply(lambda x: x['predicted_times'])
    for i in range(5):
        df[f'predicted_times_{i}'] = predicted_times.apply(lambda x: x[i] if i < len(x) else None)

    final_df = df[['clip_uid', 'annotation_uid', 'query_idx',
                   'predicted_times_0', 'predicted_times_1',
                   'predicted_times_2', 'predicted_times_3',
                   'predicted_times_4']]

    file_name = os.path.splitext(os.path.basename(file_path))[0]

    csv_file_path = f'{file_name}.csv'

    final_df.to_csv(csv_file_path, index=False)


    return csv_file_path


In [ ]:
## Ensemble.py ##

import statistics
import numpy as np
import pandas as pd
import json
from MIoU import calculate_iou, calculate_miou
from vsl_pred_to_csv import json_to_csv
from update_json_with_intervals import update_json_with_intervals
import re
import csv


## Extraction of the list of true intervals ##

def extract_true_intervals(file_path):
    # Load the CSV file into a pandas DataFrame
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
    except Exception as e:
        print(f"Error loading the CSV file: {str(e)}")

    # Verify that the DataFrame has been loaded correctly
    if 'exact_time' in df.columns:
        exact_times_column = df['exact_time']
        # print(exact_times_column)
    else:
        print("The column 'exact_time' is not present in the DataFrame.")

    ## Convert the list of strings "[x, y]" into a list of lists ##
    import json

    # Convert strings to lists of floats
    exact_times_column = [json.loads(s) for s in exact_times_column]

    return exact_times_column



########################################################################################################################

def extract_5_predictions(val_pred_csv):
    # Load the CSV file into a pandas DataFrame
    try:
        df = pd.read_csv(val_pred_csv)  # Replace 'nome_file.csv' with the correct path to your CSV file
    except FileNotFoundError:
        print("File not found.")
        exit()
    except Exception as e:
        print(f"Error loading the CSV file: {str(e)}")
        exit()

    # Ensure the columns exist in the DataFrame
    columns_to_extract = ['predicted_times_0', 'predicted_times_1', 'predicted_times_2', 'predicted_times_3',
                          'predicted_times_4']
    for col in columns_to_extract:
        if col not in df.columns:
            print(f"The column '{col}' does not exist in the DataFrame.")
            exit()

    # Extract the columns as separate lists
    predicted_times_0 = df['predicted_times_0'].tolist()
    predicted_times_1 = df['predicted_times_1'].tolist()
    predicted_times_2 = df['predicted_times_2'].tolist()
    predicted_times_3 = df['predicted_times_3'].tolist()
    predicted_times_4 = df['predicted_times_4'].tolist()

    ## Convert the list of strings "[x, y]" into a list of lists ##
    ## 0 ##
    predicted_times_0 = [json.loads(s) for s in predicted_times_0]

    ## 1 ##
    predicted_times_1 = [json.loads(s) for s in predicted_times_1]

    ## 2 ##
    predicted_times_2 = [json.loads(s) for s in predicted_times_2]

    ## 3 ##
    predicted_times_3 = [json.loads(s) for s in predicted_times_3]

    ## 4 ##
    predicted_times_4 = [json.loads(s) for s in predicted_times_4]

    return predicted_times_0, predicted_times_1, predicted_times_2, predicted_times_3, predicted_times_4


########################################################################################################################


def ensemble(num_mod=10, split=0.67, rank=1, inter=0.3, miou_choice=2, weight_choice="rank_inter",
             influent_threshold=0.67, power=1, max_weight=False, comparison="all"):
    """
    Function to perform ensemble of models.

    Parameters:
    num_mod (int): Number of models to ensemble.
    epochs (int): Number of epochs for training.
    num_ep (int): Number of epochs to consider.
    split (float): Split ratio for training and test.
    rank (int): Rank in the Rank/Intersection measure.
    inter (float): Threshold of IoU considered between intervals.
    miou_choice (int): Choice of the mean Intersection over Union (mIoU) considered.
    weight_choice (str): Measure for choosing weights.
    influent_threshold (float): Threshold for considering influential models.
    power (int): Power parameter for weighting.
    max_weight (bool): Whether to use the maximum weight as reference or not.
    comparison (str): Comparison with the presumed best or with the actual (aggregated) best.

    Returns:
    The evaluation of the model specified in the "comparison" input, the evalutaion of the ensemble and its predictions.
    """

    ########################################################################################################################
    ## Cutting non-influential models for ensemble and voting ##

    def cut_below_threshold(lista, threshold=0.67):
        """
        Cuts values in the list that are below a certain threshold relative to the maximum value.

        Parameters:
        lista: List of numerical values.
        threshold: A float representing the fraction of the maximum value below which items are set to 0.

        Returns:
        A new list where values below the threshold * max_value are set to 0.
        """
        max_value = max(lista)  # Find the maximum value in the list
        threshold_value = threshold * max_value  # Calculate the threshold value as a fraction of the max value
        return [x if x >= threshold_value else 0 for x in lista]  # Replace values below the threshold with 0

    ########################################################################################################################
    ## Function that evaluates if the intersection of two intervals is non-empty or if it is generally smaller than a certain threshold ##

    def non_empty_intersection(interval1, interval2, threshold=0.3):
        """
        Determines if the intersection of two two-dimensional intervals is non-empty and if
        the ratio between the intersection and the union is greater than a certain threshold.

        Parameters:
        interval1: List with a pair of coordinates [x1_min, x1_max]
        interval2: List with a pair of coordinates [x2_min, x2_max]
        threshold: The threshold below which the function returns 0

        Returns:
        1 if the ratio between intersection and union is greater than the threshold, otherwise 0.
        """

        x1_min, x1_max = interval1[0], interval1[1]
        x2_min, x2_max = interval2[0], interval2[1]

        # Calculate the intersection limits
        intersection_min = max(x1_min, x2_min)
        intersection_max = min(x1_max, x2_max)

        # If there is no intersection, return 0
        if intersection_min >= intersection_max:
            return 0

        # Calculate the length of the intersection and the union
        intersection_length = intersection_max - intersection_min
        union_length = (x1_max - x1_min) + (x2_max - x2_min) - intersection_length

        # Calculate the intersection/union ratio
        ratio = intersection_length / union_length

        # Return 1 if the ratio is greater than the threshold, otherwise 0
        if ratio > threshold:
            return 1
        else:
            return 0

    #########################################################################################################################
    ## Function that determines the representative for a given interval through a voting system ##

    def determine_reference(interval_list, weight_list, inter_threshold):
        """
        Determines the representative for a given interval through a voting system.

        Parameters:
        interval_list: List of intervals.
        weight_list: List of weights corresponding to the intervals.
        inter_threshold: The threshold for determining if intervals intersect.

        Returns:
        A list where each entry represents the number of votes each interval received.
        """
        votes = []  # List to store the number of votes for each interval
        non_influential_sets = {i for i, weight in enumerate(weight_list) if
                                weight == 0}  # Indices of non-influential sets

        for idx1, interval1 in enumerate(interval_list):
            if idx1 in non_influential_sets:  # Skip non-influential intervals
                votes.append(0)  # If interval is non-influential, append 0 votes
                continue

            vote_count = 0
            for idx2, interval2 in enumerate(interval_list):
                if idx2 in non_influential_sets:  # Skip non-influential intervals
                    continue

                if non_empty_intersection(interval1, interval2, inter_threshold) == 1:  # Check if intervals intersect
                    vote_count += 1  # Increment vote count for intersecting intervals

            votes.append(vote_count)  # Append the total votes for the current interval

        return votes  # Return the list of votes for all intervals

    ########################################################################################################################
    ## Convex Combination Function ##

    def convex_combination(pred_list, w, inter, max_weight=False):
        """
        Linearly combines the two-dimensional lists only if their intersection with the reference list is non-empty.

        Parameters:
        pred_list: List of two-dimensional lists.
        w: List of weights.
        threshold: The threshold for the intersection/union ratio.

        Returns:
        A linearly combined list.
        """
        new_interval = [0, 0]  # Initialize a new interval

        if max_weight:
            reference_list = pred_list[w.index(max(w))]  # The reference list is the one with the highest weight
        else:
            # Determine the reference list based on voting system
            votes = determine_reference(pred_list, w, inter)
            max_votes = max(votes)
            max_vote_indices = [i for i, val in enumerate(votes) if val == max_votes]
            selected_index = max(max_vote_indices, key=lambda i: w[i])
            reference_list = pred_list[selected_index]

        sum_weights = 0

        # Iterate over each interval in pred_list
        for i, interval in enumerate(pred_list):
            # Calculate weight intersection
            weight_intersection = non_empty_intersection(reference_list, interval, inter)

            # Weighted interval calculation
            weighted_interval = [w[i] * weight_intersection * end for end in interval]

            # Update new_interval by adding weighted_interval
            new_interval = [end1 + end2 for end1, end2 in zip(new_interval, weighted_interval)]

            # Accumulate sum of weights
            sum_weights += w[i] * weight_intersection

        # Normalize the new_interval if sum_weights is non-zero
        if sum_weights != 0:
            return [end / sum_weights for end in new_interval]
        else:
            return -1  # Return -1 if sum_weights is zero (no valid combination found)

    #########################################################################################################################
    def iou(interval1, interval2):
        """
        Calculates the Intersection over Union (IoU) of two intervals.
        Each interval is represented as a tuple (start, end).

        Parameters:
        interval1: Tuple representing the first interval (start1, end1).
        interval2: Tuple representing the second interval (start2, end2).

        Returns:
        IoU (Intersection over Union) value between the two intervals.
        """
        start1, end1 = interval1  # Unpack interval1 into start1 and end1
        start2, end2 = interval2  # Unpack interval2 into start2 and end2

        # Calculate intersection
        inter_start = max(start1, start2)  # Start of the intersection
        inter_end = min(end1, end2)  # End of the intersection
        if inter_start >= inter_end:
            inter_len = 0  # If no intersection, length is 0
        else:
            inter_len = inter_end - inter_start  # Length of the intersection

        # Calculate union
        union_len = (end1 - start1) + (end2 - start2) - inter_len  # Length of the union

        # Calculate IoU (Intersection over Union)
        if union_len == 0:
            IoU = 0  # If union length is zero, IoU is zero to avoid division by zero
        else:
            IoU = inter_len / union_len  # Otherwise, compute IoU

        return IoU

    #########################################################################################################################

    def check_intervals(true_interval, predicted_intervals, n=5, threshold=0.3):
        """
        Checks if at least one of the first n predicted intervals has IoU > threshold.

        true_interval: Tuple (start, end) representing the true interval.
        predicted_intervals: List of 5 tuples, each representing a predicted interval.
        n: Number of predicted intervals to consider (from 1 to 5).
        threshold: IoU threshold value (from 0 to 1).

        Returns 1 if at least one of the first n intervals has IoU > threshold, otherwise 0.
        """
        for k in range(0, n):
            if iou(true_interval, predicted_intervals[k]) > threshold:
                return 1
        return 0

    ########################################################################################################################
    file_path = 'VALIDATION.csv'
    exact_times = extract_true_intervals(file_path=file_path)

    file_list = []
    for j in range(num_mod):
        file_list.append(f"model_{j}.json")

    predicted_lists = {}
    keys_list = []
    for i, file in enumerate(file_list):
        val_pred = json_to_csv(file)
        key = file[:-5]  # Remove the '.json' extension from the filename to use as key
        keys_list.append(key)
        predicted_lists[key] = extract_5_predictions(val_pred_csv=val_pred)

    ########################################################################################################################
    ## Train set and test set division ##

    length = len(exact_times)
    split_point = round(length * split)

    if split != 1:
        test_exact_times = exact_times[split_point:]

        # Loop for test set
        test_predicted_lists = {}
        for key in keys_list:
            test_predicted_lists[key] = []
            for i in range(len(predicted_lists[key])):
                test_predicted_lists[key].append(predicted_lists[key][i][split_point:])

    exact_times = exact_times[:split_point]

    # Loop for train set
    train_predicted_lists = {}
    for key in keys_list:
        train_predicted_lists[key] = []
        for i in range(len(predicted_lists[key])):
            train_predicted_lists[key].append(predicted_lists[key][i][:split_point])

    ########################################################################################################################
    ## EVALUATION of old model predictions for the WEIGHTS of CONVEX COMBINATION ##
    ########################################################################################################################

    w = []  # <- list of weights

    # Dictionary definition #
    if weight_choice == "miou":
        miou_dict = {
            'mean_vec_miou_1': None,
            'mean_vec_miou_2': None,
            'mean_vec_miou_3': None,
            'mean_vec_miou': None
        }

    for j in range(num_mod):  # Model
        # print(f"\n#####Related to model: {j}#####\n")
        ################################################################################################################
        ## MIoU ##

        if weight_choice == "miou":
            vec_miou_1 = []
            vec_miou_3 = []
            vec_miou_5 = []
            vec_miou = []

            ## 1 MIoU ##
            for i in range(len(exact_times)):
                ## MIoU computation ##
                vec_miou_1.append(calculate_miou(exact_times[i], [train_predicted_lists[f"model_{j}"][0][i]]))
            ########################################################################################################################
            ## 3 MIoU ##
            for i in range(len(exact_times)):
                ## MIoU computation ##
                vec_miou_3.append(calculate_miou(exact_times[i], [train_predicted_lists[f"model_{j}"][0][i],
                                                                  train_predicted_lists[f"model_{j}"][1][i],
                                                                  train_predicted_lists[f"model_{j}"][2][i]]))
            ########################################################################################################################
            ## 5 MIoU ##
            for i in range(len(exact_times)):
                ## MIoU computation ##
                vec_miou_5.append(calculate_miou(exact_times[i], [train_predicted_lists[f"model_{j}"][0][i],
                                                                  train_predicted_lists[f"model_{j}"][1][i],
                                                                  train_predicted_lists[f"model_{j}"][2][i],
                                                                  train_predicted_lists[f"model_{j}"][3][i],
                                                                  train_predicted_lists[f"model_{j}"][4][i]]))
            ########################################################################################################################
            ## TOTAL MIoU ##
            for i in range(len(exact_times)):
                ## MIoU computation ##
                vec_miou.append(calculate_miou(exact_times[i], [train_predicted_lists[f"model_{j}"][0][i]]))
                vec_miou.append(calculate_miou(exact_times[i], [train_predicted_lists[f"model_{j}"][1][i]]))
                vec_miou.append(calculate_miou(exact_times[i], [train_predicted_lists[f"model_{j}"][2][i]]))
                vec_miou.append(calculate_miou(exact_times[i], [train_predicted_lists[f"model_{j}"][3][i]]))
                vec_miou.append(calculate_miou(exact_times[i], [train_predicted_lists[f"model_{j}"][4][i]]))

            average_miou = sum(vec_miou) / len(vec_miou) if vec_miou else 0

            ########################################################################################################################
            # Mean of miou #
            miou_dict[f'mean_vec_miou_{j}'] = [np.mean(vec_miou_1), np.mean(vec_miou_3), np.mean(vec_miou_5),
                                               average_miou]
            ########################################################################################################################
            # Adding the weight
            w.append(miou_dict[f'mean_vec_miou_{j}'][miou_choice])

        ################################################################################################################
        ## Rank, inter (%) ##

        elif weight_choice == "rank_inter":
            summation = 0
            for i in range(len(exact_times)):
                predicted_intervals = [train_predicted_lists[f"model_{j}"][0][i],
                                       train_predicted_lists[f"model_{j}"][1][i],
                                       train_predicted_lists[f"model_{j}"][2][i],
                                       train_predicted_lists[f"model_{j}"][3][i],
                                       train_predicted_lists[f"model_{j}"][4][i]]
                summation += check_intervals(exact_times[i], predicted_intervals=predicted_intervals, n=rank,
                                             threshold=inter)

            w.append(summation / len(exact_times) * 100)

        else:
            w.append(1)

    ########################################################################################################################
    ## ENSEMBLE ##
    ########################################################################################################################

    w = cut_below_threshold(w, threshold=influent_threshold)

    w = [x ** power for x in w]

    ########################################################################################################################
    ## CONSTRUCTION OF THE ENSEMBLE FOR THE TRAIN ##
    if split == 1:

        train_pred_models = []
        for i in range(5):
            train_models_i = []
            for j in range(num_mod):
                train_models_i.append(train_predicted_lists[f"model_{j}"][i])

            train_pred_models.append(train_models_i)

        pred_0_ens = [
            convex_combination(sublists, w, inter=inter, max_weight=max_weight) for sublists in
            zip(*train_pred_models[0])
        ]
        pred_1_ens = [
            convex_combination(sublists, w, inter=inter, max_weight=max_weight) for sublists in
            zip(*train_pred_models[1])
        ]
        pred_2_ens = [
            convex_combination(sublists, w, inter=inter, max_weight=max_weight) for sublists in
            zip(*train_pred_models[2])
        ]
        pred_3_ens = [
            convex_combination(sublists, w, inter=inter, max_weight=max_weight) for sublists in
            zip(*train_pred_models[3])
        ]
        pred_4_ens = [
            convex_combination(sublists, w, inter=inter, max_weight=max_weight) for sublists in
            zip(*train_pred_models[4])
        ]

        ############################################################################################################################
        ## COMBINED LIST, CONTAINS THE PREDICTION IN MATRIX FORM.

        combined_list = [
            [pred_0_ens[i], pred_1_ens[i], pred_2_ens[i], pred_3_ens[i], pred_4_ens[i]]
            for i in range(len(pred_0_ens))
        ]

        ####################################################################################################################
        ## EVALUATION ENSAMBLE ON TRAIN SET (just when it is needed the perfomance on the whole dataset)
        ####################################################################################################################
        ## MIoU ##

        if weight_choice == "miou":
            vec_miou_1 = []
            vec_miou_3 = []
            vec_miou_5 = []
            vec_miou = []

            ## 1 MIoU ##
            for i in range(len(exact_times)):
                ## MIoU computation ##
                vec_miou_1.append(calculate_miou(exact_times[i], [pred_0_ens[i]]))
            ########################################################################################################################
            ## 3 MIoU ##
            for i in range(len(exact_times)):
                ## MIoU computation ##
                vec_miou_3.append(calculate_miou(exact_times[i], [pred_0_ens[i], pred_1_ens[i], pred_2_ens[i]]))
            ########################################################################################################################
            ## 5 MIoU ##
            for i in range(len(exact_times)):
                ## MIoU computation ##
                vec_miou_5.append(
                    calculate_miou(exact_times[i],
                                   [pred_0_ens[i], pred_1_ens[i], pred_2_ens[i], pred_3_ens[i], pred_4_ens[i]]))
            ########################################################################################################################
            ## TOTAL MIoU ##

            for i in range(len(exact_times)):
                ## MIoU computation ##
                vec_miou.append(calculate_miou(exact_times[i], [pred_0_ens[i]]))
                vec_miou.append(calculate_miou(exact_times[i], [pred_1_ens[i]]))
                vec_miou.append(calculate_miou(exact_times[i], [pred_2_ens[i]]))
                vec_miou.append(calculate_miou(exact_times[i], [pred_3_ens[i]]))
                vec_miou.append(calculate_miou(exact_times[i], [pred_4_ens[i]]))

            average_miou = sum(vec_miou) / len(vec_miou) if vec_miou else 0
            ########################################################################################################################
            # Mean of miou #
            ens_vec_miou = [np.mean(vec_miou_1), np.mean(vec_miou_3), np.mean(vec_miou_5), average_miou]

        ####################################################################################################################
        ## Rank, inter (%) ##

        if weight_choice == "rank_inter":
            summation = 0
            for i in range(len(exact_times)):
                predicted_intervals = [pred_0_ens[i],
                                       pred_1_ens[i],
                                       pred_2_ens[i],
                                       pred_3_ens[i],
                                       pred_4_ens[i]]

                summation += check_intervals(exact_times[i], predicted_intervals=predicted_intervals, n=rank,
                                             threshold=inter)

            rank_int_ens = summation / len(exact_times) * 100

        ########################################################################################################################
        ## EVALUATION ON THE TRAIN SET OF THE ORIGINAL MODELS TO COMPARE THEM WITH ENSEMBLE##
        ########################################################################################################################

        # Dictionary #
        if weight_choice == "miou":
            miou_dict = {
                'mean_vec_miou_1': None,
                'mean_vec_miou_2': None,
                'mean_vec_miou_3': None,
                'mean_vec': None
            }

        elif weight_choice == "rank_inter":
            rank_inter_dict = {
            }

        for j in range(num_mod):  # Models
            # print(f"\n#####Related to models: {j}#####\n")

            if weight_choice == "miou":
                vec_miou_1 = []
                vec_miou_3 = []
                vec_miou_5 = []
                vec_miou = []

                ## 1 MIoU ##
                for i in range(len(exact_times)):
                    ## MIoU computation ##
                    vec_miou_1.append(
                        calculate_miou(exact_times[i], [train_predicted_lists[f"model_{j}"][0][i]]))
                ########################################################################################################################
                ## 3 MIoU ##
                for i in range(len(exact_times)):
                    ## MIoU computation ##
                    vec_miou_3.append(
                        calculate_miou(exact_times[i], [train_predicted_lists[f"model_{j}"][0][i],
                                                        train_predicted_lists[f"model_{j}"][1][i],
                                                        train_predicted_lists[f"model_{j}"][2][
                                                            i]]))
                ########################################################################################################################
                ## 5 MIoU ##
                for i in range(len(exact_times)):
                    ## MIoU computation ##
                    vec_miou_5.append(
                        calculate_miou(exact_times[i], [train_predicted_lists[f"model_{j}"][0][i],
                                                        train_predicted_lists[f"model_{j}"][1][i],
                                                        train_predicted_lists[f"model_{j}"][2][i],
                                                        train_predicted_lists[f"model_{j}"][3][i],
                                                        train_predicted_lists[f"model_{j}"][4][
                                                            i]]))
                ########################################################################################################################
                ## TOTAL MIoU ##
                for i in range(len(exact_times)):
                    ## MIoU computation ##
                    vec_miou.append(calculate_miou(exact_times[i], [train_predicted_lists[f"model_{j}"][0][i]]))
                    vec_miou.append(calculate_miou(exact_times[i], [train_predicted_lists[f"model_{j}"][1][i]]))
                    vec_miou.append(calculate_miou(exact_times[i], [train_predicted_lists[f"model_{j}"][2][i]]))
                    vec_miou.append(calculate_miou(exact_times[i], [train_predicted_lists[f"model_{j}"][3][i]]))
                    vec_miou.append(calculate_miou(exact_times[i], [train_predicted_lists[f"model_{j}"][4][i]]))

                average_miou = sum(vec_miou) / len(vec_miou) if vec_miou else 0

                ########################################################################################################################
                # Mean of miou #
                miou_dict[f'mean_vec_miou_{j}'] = [np.mean(vec_miou_1), np.mean(vec_miou_3), np.mean(vec_miou_5),
                                                   average_miou]

            elif weight_choice == "rank_inter":
                summation = 0
                for i in range(len(exact_times)):
                    summation += check_intervals(exact_times[i], [train_predicted_lists[f"model_{j}"][0][i],
                                                                  train_predicted_lists[f"model_{j}"][1][i],
                                                                  train_predicted_lists[f"model_{j}"][2][i],
                                                                  train_predicted_lists[f"model_{j}"][3][i],
                                                                  train_predicted_lists[f"model_{j}"][4][i]],
                                                 n=rank,
                                                 threshold=inter)
                rank_inter_dict[f'rank_inter_{j}'] = summation / len(exact_times) * 100

        if comparison == "best":
            best_list = []  # Fictitious
            ########################################################################################################################
            ## PRINT OF THE RESULTS OBTAINED FROM THE SUPPOSED BEST AND COMPARE THEM WITH THE ENSEMBLE ##

            # Step 1: Find the index of the largest value in the list
            max_index = w.index(max(w))

            ## MMIoU ##
            if weight_choice == "miou":
                # Step 2: Get the corresponding key from the dictionary
                max_key = list(miou_dict.keys())[max_index]

                # Step 3: Retrieve the value from the dictionary using the key
                best_miou = miou_dict[max_key]

                print(
                    f"The results of the supposed best (w.r.t. {weight_choice}) for the MMIoU  1, 3, 5 and total respectively are: {best_miou[0]}, {best_miou[1]}, {best_miou[2]}, {best_miou[3]}")
                print(
                    f"The results given by the Ensemble are:{ens_vec_miou[0]}, {ens_vec_miou[1]}, {ens_vec_miou[2]}, {ens_vec_miou[3]}")

                return best_miou, ens_vec_miou, combined_list, best_list

            ## Rank, inter (%) ##
            if weight_choice == "rank_inter":
                # Step 2: Get the corresponding key from the dictionary
                max_key = list(rank_inter_dict.keys())[max_index]

                # Step 3: Retrieve the value from the dictionary using the key
                best_rank_inter = rank_inter_dict[max_key]

                print(
                    f"The best (w.r.t. {weight_choice}) single-model result obtained with rank: {rank}, IoU: {inter} (%) was: {best_rank_inter}. \nThe Ensemble gave back: {rank_int_ens}.")

                return best_rank_inter, rank_int_ens, combined_list, best_list

        elif comparison == "all":
            best_list = []  # Fictitious
            ########################################################################################################################
            ## PRINT OF THE BEST RESULTS OBTAINED AND COMPARE THEM WITH THE ENSEMBLE ##

            ## MMIoU ##
            if weight_choice == "miou":

                max_first = float('-inf')
                max_second = float('-inf')
                max_third = float('-inf')
                max_fourth = float('-inf')

                for values in miou_dict.values():
                    if values is not None:
                        if values[0] > max_first:
                            max_first = values[0]
                        if values[1] > max_second:
                            max_second = values[1]
                        if values[2] > max_third:
                            max_third = values[2]
                        if values[3] > max_fourth:
                            max_fourth = values[3]

                maximum = [max_first, max_second, max_third, max_fourth]
                print(
                    f"The aggregate best results for the MMIoU  1, 3 and 5 respectively are: {maximum[0]}, {maximum[1]}, {maximum[2]}, {maximum[3]}")
                print(
                    f"The results given by the Ensemble are:{ens_vec_miou[0]}, {ens_vec_miou[1]}, {ens_vec_miou[2]}, {ens_vec_miou[3]}")

                return maximum, ens_vec_miou, combined_list, best_list

            ## Rank, inter (%) ##
            if weight_choice == "rank_inter":
                max_value = 0
                for value in rank_inter_dict.values():
                    if not isinstance(value, list):
                        if value > max_value:
                            max_value = value
                print(
                    f"The aggregate best single-model result obtained with rank: {rank}, IoU: {inter} (%) was: {max_value}. \nThe Ensemble gave back: {rank_int_ens}.")

                return max_value, rank_int_ens, combined_list, best_list
    ########################################################################################################################
    ## CONSTRUCTION OF THE ENSEMBLE FOR THE TEST ##
    elif split != 1:

        test_pred_models = []
        for i in range(5):
            test_models_i = []
            for j in range(num_mod):
                test_models_i.append(test_predicted_lists[f"model_{j}"][i])

            test_pred_models.append(test_models_i)

        pred_0_ens = [
            convex_combination(sublists, w, inter=inter, max_weight=max_weight) for sublists in
            zip(*test_pred_models[0])
        ]
        pred_1_ens = [
            convex_combination(sublists, w, inter=inter, max_weight=max_weight) for sublists in
            zip(*test_pred_models[1])
        ]
        pred_2_ens = [
            convex_combination(sublists, w, inter=inter, max_weight=max_weight) for sublists in
            zip(*test_pred_models[2])
        ]
        pred_3_ens = [
            convex_combination(sublists, w, inter=inter, max_weight=max_weight) for sublists in
            zip(*test_pred_models[3])
        ]
        pred_4_ens = [
            convex_combination(sublists, w, inter=inter, max_weight=max_weight) for sublists in
            zip(*test_pred_models[4])
        ]

        ############################################################################################################################
        ## COMBINED LIST, CONTAINS THE PREDICTION OF ENSEMBLE IN MATRIX FORM.

        combined_list = [
            [pred_0_ens[i], pred_1_ens[i], pred_2_ens[i], pred_3_ens[i], pred_4_ens[i]]
            for i in range(len(pred_0_ens))
        ]

        ########################################################################################################################
        ## EVALUATION ENSEMBLE ON TEST ##
        ########################################################################################################################
        ## MIOU ##

        if weight_choice == "miou":
            vec_miou_1 = []
            vec_miou_3 = []
            vec_miou_5 = []
            vec_miou = []

            ## 1 MIoU ##
            for i in range(len(test_exact_times)):
                ## MIoU computation ##
                vec_miou_1.append(calculate_miou(test_exact_times[i], [pred_0_ens[i]]))
            ########################################################################################################################
            ## 3 MIoU ##
            for i in range(len(test_exact_times)):
                ## MIoU computation ##
                vec_miou_3.append(calculate_miou(test_exact_times[i], [pred_0_ens[i], pred_1_ens[i], pred_2_ens[i]]))
            ########################################################################################################################
            ## 5 MIoU ##
            for i in range(len(test_exact_times)):
                ## MIoU computation ##
                vec_miou_5.append(calculate_miou(test_exact_times[i],
                                                 [pred_0_ens[i], pred_1_ens[i], pred_2_ens[i], pred_3_ens[i],
                                                  pred_4_ens[i]]))
            ########################################################################################################################
            ## TOTAL MIoU ##

            for i in range(len(test_exact_times)):
                ## MIoU computation ##
                vec_miou.append(calculate_miou(test_exact_times[i], [pred_0_ens[i]]))
                vec_miou.append(calculate_miou(test_exact_times[i], [pred_1_ens[i]]))
                vec_miou.append(calculate_miou(test_exact_times[i], [pred_2_ens[i]]))
                vec_miou.append(calculate_miou(test_exact_times[i], [pred_3_ens[i]]))
                vec_miou.append(calculate_miou(test_exact_times[i], [pred_4_ens[i]]))

            average_miou = sum(vec_miou) / len(vec_miou) if vec_miou else 0

            ########################################################################################################################
            # Mean of miou #
            ens_vec_miou = [np.mean(vec_miou_1), np.mean(vec_miou_3), np.mean(vec_miou_5), average_miou]

        ####################################################################################################################
        ## Rank, inter (%) ##

        if weight_choice == "rank_inter":
            summation = 0

            for i in range(len(test_exact_times)):
                predicted_intervals = [pred_0_ens[i],
                                       pred_1_ens[i],
                                       pred_2_ens[i],
                                       pred_3_ens[i],
                                       pred_4_ens[i]]

                summation += check_intervals(test_exact_times[i], predicted_intervals=predicted_intervals, n=rank,
                                             threshold=inter)

            rank_int_ens = summation / len(test_exact_times) * 100

        ############################################################################################################################
        ## BEST LIST, CONTAINS THE PREDICTION OF THE SUPPOSED BEST MODEL IN MATRIX FORM.
        max_index = w.index(max(w))

        best_list = [
            [test_predicted_lists[f"model_{max_index}"][0][i], test_predicted_lists[f"model_{max_index}"][1][i], test_predicted_lists[f"model_{max_index}"][2][i], test_predicted_lists[f"model_{max_index}"][3][i], test_predicted_lists[f"model_{max_index}"][4][i]]
            for i in range(len(test_predicted_lists[f"model_{max_index}"][0]))
        ]

        ########################################################################################################################
        ## EVALUATION ON THE TEST SET OF THE ORIGINAL MODEL TO COMPARE THEM WITH ENSEMBLE##
        ########################################################################################################################

        # Dictionary #
        if weight_choice == "miou":
            miou_dict = {
                'mean_vec_miou_1': None,
                'mean_vec_miou_2': None,
                'mean_vec_miou_3': None,
                'mean_vec_miou': None
            }

        elif weight_choice == "rank_inter":
            rank_inter_dict = {
            }

        for j in range(num_mod):  # Models
            # print(f"\n#####Related to models: {j}#####\n")
            if weight_choice == "miou":
                vec_miou_1 = []
                vec_miou_3 = []
                vec_miou_5 = []
                vec_miou = []

                ## 1 MIoU ##
                for i in range(len(test_exact_times)):
                    ## MIoU computation ##
                    vec_miou_1.append(
                        calculate_miou(test_exact_times[i], [test_predicted_lists[f"model_{j}"][0][i]]))
                ########################################################################################################################
                ## 3 MIoU ##
                for i in range(len(test_exact_times)):
                    ## MIoU computation ##
                    vec_miou_3.append(calculate_miou(test_exact_times[i], [test_predicted_lists[f"model_{j}"][0][i],
                                                                           test_predicted_lists[f"model_{j}"][1][i],
                                                                           test_predicted_lists[f"model_{j}"][2][
                                                                               i]]))
                ########################################################################################################################
                ## 5 MIoU ##
                for i in range(len(test_exact_times)):
                    ## MIoU computation ##
                    vec_miou_5.append(calculate_miou(test_exact_times[i], [test_predicted_lists[f"model_{j}"][0][i],
                                                                           test_predicted_lists[f"model_{j}"][1][i],
                                                                           test_predicted_lists[f"model_{j}"][2][i],
                                                                           test_predicted_lists[f"model_{j}"][3][i],
                                                                           test_predicted_lists[f"model_{j}"][4][
                                                                               i]]))
                ########################################################################################################################
                ## TOTAL MIoU ##
                for i in range(len(test_exact_times)):
                    ## MIoU computation ##
                    vec_miou.append(calculate_miou(test_exact_times[i], [test_predicted_lists[f"model_{j}"][0][i]]))
                    vec_miou.append(calculate_miou(test_exact_times[i], [test_predicted_lists[f"model_{j}"][1][i]]))
                    vec_miou.append(calculate_miou(test_exact_times[i], [test_predicted_lists[f"model_{j}"][2][i]]))
                    vec_miou.append(calculate_miou(test_exact_times[i], [test_predicted_lists[f"model_{j}"][3][i]]))
                    vec_miou.append(calculate_miou(test_exact_times[i], [test_predicted_lists[f"model_{j}"][4][i]]))

                average_miou = sum(vec_miou) / len(vec_miou) if vec_miou else 0

                ########################################################################################################################
                # Mean of miou #
                miou_dict[f'mean_vec_miou_{j}'] = [np.mean(vec_miou_1), np.mean(vec_miou_3), np.mean(vec_miou_5),
                                                   average_miou]

            elif weight_choice == "rank_inter":
                summation = 0
                for i in range(len(test_exact_times)):
                    summation += check_intervals(test_exact_times[i], [test_predicted_lists[f"model_{j}"][0][i],
                                                                       test_predicted_lists[f"model_{j}"][1][i],
                                                                       test_predicted_lists[f"model_{j}"][2][i],
                                                                       test_predicted_lists[f"model_{j}"][3][i],
                                                                       test_predicted_lists[f"model_{j}"][4][i]],
                                                 n=rank,
                                                 threshold=inter)
                rank_inter_dict[f'rank_inter_{j}'] = summation / len(test_exact_times) * 100

                # print(f"rank: {rank}, IoU: {inter} (%)", rank_inter_dict[f'rank_inter_{j}_{r}'])

        if comparison == "best":
            ########################################################################################################################
            ## PRINT OF THE RESULTS OBTAINED FROM THE SUPPOSED BEST AND COMPARE THEM WITH THE ENSEMBLE ##

            # Step 1: Find the index of the largest value in the list
            max_index = w.index(max(w))

            ## MMIoU ##
            if weight_choice == "miou":
                # Step 2: Get the corresponding key from the dictionary
                max_key = list(miou_dict.keys())[max_index]

                # Step 3: Retrieve the value from the dictionary using the key
                best_miou = miou_dict[max_key]

                print(
                    f"The results of the supposed best (w.r.t. {weight_choice}) for the MMIoU  1, 3, 5 and total respectively are: {best_miou[0]}, {best_miou[1]}, {best_miou[2]}, {best_miou[3]}")
                print(
                    f"The results given by the Ensemble are:{ens_vec_miou[0]}, {ens_vec_miou[1]}, {ens_vec_miou[2]}, {ens_vec_miou[3]}")

                return best_miou, ens_vec_miou, combined_list, best_list

            ## Rank, inter (%) ##
            if weight_choice == "rank_inter":
                # Step 2: Get the corresponding key from the dictionary
                max_key = list(rank_inter_dict.keys())[max_index]

                # Step 3: Retrieve the value from the dictionary using the key
                best_rank_inter = rank_inter_dict[max_key]

                print(
                    f"The best (w.r.t. {weight_choice}) single-model result obtained with rank: {rank}, IoU: {inter} (%) was: {best_rank_inter}. \nThe Ensemble gave back: {rank_int_ens}.")

                return best_rank_inter, rank_int_ens, combined_list, best_list

        elif comparison == "all":
            best_list = []  # Fictitious
            ########################################################################################################################
            ## PRINT OF THE BEST RESULTS OBTAINED AND COMPARE THEM WITH THE ENSEMBLE ##

            ## MMIoU ##
            if weight_choice == "miou":

                max_first = float('-inf')
                max_second = float('-inf')
                max_third = float('-inf')
                max_fourth = float('-inf')

                for values in miou_dict.values():
                    if values is not None:
                        if values[0] > max_first:
                            max_first = values[0]
                        if values[1] > max_second:
                            max_second = values[1]
                        if values[2] > max_third:
                            max_third = values[2]
                        if values[3] > max_fourth:
                            max_fourth = values[3]

                maximum = [max_first, max_second, max_third, max_fourth]
                print(
                    f"The aggregate best results for the MMIoU  1, 3 and 5 respectively are: {maximum[0]}, {maximum[1]}, {maximum[2]}, {maximum[3]}")
                print(
                    f"The results given by the Ensemble are:{ens_vec_miou[0]}, {ens_vec_miou[1]}, {ens_vec_miou[2]}, {ens_vec_miou[3]}")

                return maximum, ens_vec_miou, combined_list, best_list

            ## Rank, inter (%) ##
            if weight_choice == "rank_inter":
                max_value = 0
                for value in rank_inter_dict.values():
                    if not isinstance(value, list):
                        if value > max_value:
                            max_value = value
                print(
                    f"The aggregate best single-model result obtained with rank: {rank}, IoU: {inter} (%) was: {max_value}. \nThe Ensemble gave back: {rank_int_ens}.")

                return max_value, rank_int_ens, combined_list, best_list


########################################################################################################################
## FIXED-PARAMETERS ##

num_mod = 153

split = 0.67

miou_choice = 0
########################################################################################################################
## GRID-SEARCH ##

## Important observation: Note that rank 5 as a measure of choosing the best model is significantly worse than rank 1. This can be intuitively explained by the fact that if you hypothetically have a model with all 5 predicted intervals being different, it indicates that the model was potentially not very confident in the first interval, which is the one we are most interested in. Therefore, a behavior would be positively evaluated that in reality is not. ##

hyper_p = []
grids = []
combined = []
best = []
linear_variations = []
for comparison in ["best"]:  # in ["best", "all"]
    for max_weight in [True, False]:  # True is largerly worse than the False. FINAL CHOICE: [True, False]
        for rank in [1]:  # (metric rank_inter) -> 2 worse than 5 which is worse than 1 itself. FINAL CHOICE: [1] Look up for the reason.
            for inter in [0.5, 0.3]:  # (metric miou) -> 0.1 worse than 0.3, 0.5  # (metric rank_inter) -> 0.3 gives back slightly higher peaks, but in general they show similar behaviours. FINAL CHOICE: [0.5, 0.3]
                for metric in ["miou", "rank_inter"]:  # (metric rank_inter) -> eventhough metric rank_inter is widely used, the more stable results are given by the miou. FINAL CHOICE: ["miou", "rank_inter"]
                    for influent_threshold in [0.8, 0.67]:  # (metric rank_inter) -> 0.8 worse than 0.67. FINAL CHOICE: [0.8, 0.67]
                        for power in [1, 10, 100]:  # Heuristically, 1, 10, 100.
                            print(f"###The hyper-parameters set: split: {split}, rank: {rank}, inter: {inter}, metric: {metric}, miou_choice: {miou_choice}, influent_threshold: {influent_threshold}, power: {power}, max_weight: {max_weight}, comparison: {comparison}### ")

                            hyper_p.append([split, rank, inter, metric, miou_choice, influent_threshold, power, max_weight,comparison])

                            grid = ensemble(num_mod, split, rank, inter, miou_choice, metric, influent_threshold, power, max_weight, comparison)

                            grids.append(grid)
                            if metric == "miou":
                                lin_var = (grid[1][miou_choice] - grid[0][miou_choice]) / grid[0][miou_choice] * 100
                                linear_variations.append(lin_var)
                                combined.append(
                                    [split, rank, inter, metric, miou_choice, influent_threshold, power, max_weight,
                                     comparison, lin_var, grid[2]])
                                best.append(
                                    [split, rank, inter, metric, miou_choice, influent_threshold, power, max_weight,
                                     comparison, lin_var, grid[3]])

                            elif metric == "rank_inter":
                                lin_var = (grid[1] - grid[0]) / grid[0] * 100
                                linear_variations.append(lin_var)
                                combined.append(
                                    [split, rank, inter, metric, miou_choice, influent_threshold, power, max_weight,
                                     comparison, lin_var, grid[2]])
                                best.append(
                                    [split, rank, inter, metric, miou_choice, influent_threshold, power, max_weight,
                                     comparison, lin_var, grid[3]])
                            print("\n")

########################################################################################################################
## SAVING HYPER-PARAMETERS AND PREDICTIONS INTO A CSV FILE

# Creating a DataFrame from the combined list
df_combined = pd.DataFrame(combined)
df_best = pd.DataFrame(best)

# Adding columns conditionally
if metric == "miou":
    df_combined.columns = ["split", "rank", "inter", "metric", "miou_choice", "influent_threshold", "power",
                           "max_weight", "comparison", "miou", "pred"]
    df_best.columns = ["split", "rank", "inter", "metric", "miou_choice", "influent_threshold", "power",
                           "max_weight", "comparison", "miou", "pred"]
elif metric == "rank_inter":
    df_combined.columns = ["split", "rank", "inter", "metric", "miou_choice", "influent_threshold", "power",
                           "max_weight", "comparison", "rank_inter", "pred"]
    df_best.columns = ["split", "rank", "inter", "metric", "miou_choice", "influent_threshold", "power",
                           "max_weight", "comparison", "rank_inter", "pred"]

df_combined.to_csv('df_combined.csv', index=False)
df_best.to_csv('df_best.csv', index=False)

########################################################################################################################
## WRITING ON "OUTPUT.TXT" FILE THE BEST AND THE WORST CONFIGURATIONS ##

file = "output.txt"
with open(file, 'w') as file:
    positive_values = [(index, value) for index, value in enumerate(linear_variations) if value > 0]
    top_positive = sorted(positive_values, key=lambda x: x[1], reverse=True)[:int(len(linear_variations) / 4)]

    negative_values = [(index, value) for index, value in enumerate(linear_variations) if value < 0]
    top_negative = sorted(negative_values, key=lambda x: x[1])[:int(len(linear_variations) / 4)]

    for index, value in top_positive:
        print(
            f"The hyper-parameters set: split: {hyper_p[index][0]}, rank: {hyper_p[index][1]}, inter: {hyper_p[index][2]}, metric: {hyper_p[index][3]}, miou_choice: {hyper_p[index][4]}, influent_threshold: {hyper_p[index][5]}, power: {hyper_p[index][6]}, max_weight: {hyper_p[index][7]}, comparison: {hyper_p[index][8]}###",
            file=file)
        print(
            f"results: {grids[index][0]}, {grids[index][1]}, \nlinear variations: {np.round(linear_variations[index], 4)} (%)",
            file=file)

    for index, value in top_negative:
        print(
            f"The hyper-parameters set: split: {hyper_p[index][0]}, rank: {hyper_p[index][1]}, inter: {hyper_p[index][2]}, metric: {hyper_p[index][3]}, miou_choice: {hyper_p[index][4]}, influent_threshold: {hyper_p[index][5]}, power: {hyper_p[index][6]}, max_weight: {hyper_p[index][7]}, comparison: {hyper_p[index][8]}###",
            file=file)
        print(
            f"results: {grids[index][0]}, {grids[index][1]}, \nlinear variations: {np.round(linear_variations[index], 4)} (%)",
            file=file)

########################################################################################################################
## COLOR PRINTING TO VISUALIZE THE RESULTS ##
import matplotlib.pyplot as plt
import numpy as np

values = linear_variations
# Separate positive and negative values
positive_values = np.maximum(0, linear_variations)
negative_values = np.minimum(0, linear_variations)

# Normalize positive values between 0 and 1
norm_positive_values = positive_values / positive_values.max() if positive_values.max() != 0 else positive_values

# Normalize negative values between 0 and 1
norm_negative_values = negative_values / negative_values.min() if negative_values.min() != 0 else negative_values

# Create colormaps for positive and negative values
colormap_positive = plt.get_cmap('Reds')
colormap_negative = plt.get_cmap('Blues_r')  # '_r' to get the reversed colormap

# Assign colors
colors = []
for value in linear_variations:
    if value > 0:
        color = colormap_positive(value / positive_values.max())
    elif value < 0:
        color = colormap_negative(value / negative_values.min())
    else:
        color = (1, 1, 1, 1)  # White color for zero
    colors.append(color)

# Create a figure and axis
fig, ax = plt.subplots(figsize=(15, 2))

# Remove axes and borders
ax.axis('off')

# Create the one-dimensional heatmap
for j in range(len(linear_variations)):
    ax.add_patch(plt.Rectangle((j, 0), 1, 1, color=colors[j]))

# Show the heatmap
plt.xlim(0, len(linear_variations))
plt.ylim(0, 1)

# Add a colorbar as a legend
sm_positive = plt.cm.ScalarMappable(cmap=colormap_positive, norm=plt.Normalize(vmin=0, vmax=positive_values.max()))
sm_negative = plt.cm.ScalarMappable(cmap=colormap_negative, norm=plt.Normalize(vmin=negative_values.min(), vmax=0))

# Configure and display the colorbar for positive values
cbar_positive = plt.colorbar(sm_positive, ax=ax, orientation='horizontal', pad=0.1, aspect=40, shrink=0.8)
cbar_positive.set_label('Positive Values')

# Configure and display the colorbar for negative values
cbar_negative = plt.colorbar(sm_negative, ax=ax, orientation='horizontal', pad=0.3, aspect=40, shrink=0.8)
cbar_negative.set_label('Negative Values')

plt.show()

########################################################################################################################
## READING THE PREDICTION OF THE BEST ENSEMBLE MODEL TAKEN FROM THE "OUTPUT.TXT" FILE ##

file = "output.txt"
# Open the file in read mode
with open(file, 'r') as file:
    # Read the first line
    parameters_line = file.readline()

    # The correct regex pattern
    pattern = r"split: (\d+\.\d+), rank: (\d+), inter: (\d+\.\d+), metric: (\w+), miou_choice: (\d+), influent_threshold: (\d+\.\d+), power: (\d+), max_weight: (True|False), comparison: (\w+)"

    # Search for the pattern in the string
    match = re.search(pattern, parameters_line)

    if match:
        split = float(match.group(1))
        rank = int(match.group(2))
        inter = float(match.group(3))
        metric = match.group(4)
        miou_choice = int(match.group(5))
        influent_threshold = float(match.group(6))
        power = int(match.group(7))
        max_weight = match.group(8) == "True"
        comparison = match.group(9)

        # Creating a dictionary with the parameters
        params = {
            "metric": metric,
            'split': split,
            'rank': rank,
            'inter': inter,
            'miou_choice': miou_choice,
            'influent_threshold': influent_threshold,
            'power': power,
            'max_weight': max_weight,
            'comparison': comparison
        }

        # Print the parameter dictionary
        print(params)

        # Filter the df_combined DataFrame using the parameter values

        # Filtering the DataFrame using the parameter values
        filtro = (
                (df_combined['split'] == params['split']) &
                (df_combined['rank'] == params['rank']) &
                (df_combined['inter'] == params['inter']) &
                (df_combined['miou_choice'] == params['miou_choice']) &
                (df_combined['influent_threshold'] == params['influent_threshold']) &
                (df_combined['power'] == params['power']) &
                (df_combined['max_weight'] == params['max_weight']) &
                (df_combined['comparison'] == params['comparison']) &
                (df_combined['metric'] == params['metric'])
        )

        ## Here the prediction delivered from the supposed best, for the run that built the ensemble, is selected.

        filtro_best = (
                (df_best['split'] == params['split']) &
                (df_best['rank'] == params['rank']) &
                (df_best['inter'] == params['inter']) &
                (df_best['miou_choice'] == params['miou_choice']) &
                (df_best['influent_threshold'] == params['influent_threshold']) &
                (df_best['power'] == params['power']) &
                (df_best['max_weight'] == params['max_weight']) &
                (df_best['comparison'] == params['comparison']) &
                (df_best['metric'] == params['metric'])
        )

        # Apply the filter
        df_filtered = df_combined[filtro]
        df_best_filtered = df_best[filtro_best]

        ################################################################################################################
        ## ENSEMBLE ##
        ################################################################################################################
        # Access the 'pred' column of the filtered row
        if not df_filtered.empty:
            pred_value = df_filtered.iloc[0]['pred']
            # print(pred_value)
        else:
            print("No matching rows found")

        # Create a list of dictionaries to build the DataFrame
        rows = []
        for sublist in pred_value:
            row_dict = {
                'col1': sublist[0],
                'col2': sublist[1],
                'col3': sublist[2],
                'col4': sublist[3],
                'col5': sublist[4]
            }
            rows.append(row_dict)

        # Create the DataFrame
        df_pred_value = pd.DataFrame(rows)

        # Write the DataFrame to CSV file
        csv_file_path = "predictions.csv"
        df_pred_value.to_csv(csv_file_path, index=False)

        print(f"Predictions written to {csv_file_path}")

        ################################################################################################################
        ## BEST ##
        ################################################################################################################
        # Access the 'pred' column of the filtered row
        if not df_best_filtered.empty:
            best_pred_value = df_best_filtered.iloc[0]['pred']
            # print(best_pred_value)
        else:
            print("No matching rows found")

        # Create a list of dictionaries to build the DataFrame
        rows = []
        for sublist in best_pred_value:
            row_dict = {
                'col1': sublist[0],
                'col2': sublist[1],
                'col3': sublist[2],
                'col4': sublist[3],
                'col5': sublist[4]
            }
            rows.append(row_dict)

        # Create the DataFrame
        df_best_pred_value = pd.DataFrame(rows)

        # Write the DataFrame to CSV file
        csv_file_path = "predictions_best.csv"
        df_best_pred_value.to_csv(csv_file_path, index=False)

        print(f"Predictions written to {csv_file_path}")

    else:
        print("No match found for parameters in the string.")

########################################################################################################################
## REWRITING ENTRIES OF THE SOURCE FILE TO INCLUDE ENSEMBLE PREDICTIONS FOR EVALUATION ##

percorso_file = 'VALIDATION.csv'
exact_times = extract_true_intervals(file_path=percorso_file)
length = len(exact_times)
split_point = round(length * split)

########################################################################################################################
## ENSEMBLE ##
########################################################################################################################

# Original JSON file name and backup copy
original_json_filename = 'model_0.json'
backup_json_filename = 'model_ensemble.json'

# Read content from the original JSON file
with open(original_json_filename, 'r') as original_file:
    data = json.load(original_file)

# Copy data to a new dictionary
backup_data = data.copy()

# Check if there are at least "split_point" entries in 'results'
if len(backup_data['results']) > split_point:
    # Remove the first "split_point" entries from 'results'
    backup_data['results'] = backup_data['results'][split_point:]

    # Overwrite the JSON file with updated data
    with open(backup_json_filename, 'w') as backup_file:
        json.dump(backup_data, backup_file, indent=4)
        print(f"First {split_point} entries successfully removed from 'results'.")

    ########################################################################################################################
    ## OVERWRITING THE REMAINING ENTRIES WITH THOSE PREDICTED BY THE ENSEMBLE ##

    # Load data from the CSV file
    csv_filename = 'predictions.csv'  # Replace with the actual name of your CSV file


    # Function to convert a string '[x, y]' into a list of floats [x, y]

    def string_to_list(s):
        return [float(num) for num in s.strip('[]').split(',')]


    csv_data = []
    with open(csv_filename, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)

        # Skip the first row (column headers)
        next(reader)  # This step skips the first row

        for row in reader:
            csv_data.append([string_to_list(item) for item in row])

    with open(backup_json_filename, 'r') as jsonfile:
        backup_data = json.load(jsonfile)

    # Check if there are enough rows in the CSV to update all 'predicted_times' sections
    num_csv_rows = len(csv_data)
    num_results = len(backup_data['results'])

    if num_csv_rows < num_results:
        print(
            f"Warning: The number of rows in the CSV ({num_csv_rows}) is less than the number of 'results' sections to update ({num_results}).")
        print("Operation aborted.")
    else:
        # Sequentially update the 'predicted_times' sections of 'results'
        for i in range(num_results):
            backup_data['results'][i]['predicted_times'] = csv_data[i]

        # Overwrite the JSON file with updated data
        with open(backup_json_filename, 'w') as jsonfile:
            json.dump(backup_data, jsonfile, indent=4)
            print(
                f"Update complete. The first {num_results} 'predicted_times' sections have been successfully updated.")
else:
    print("Not enough items to remove.")

########################################################################################################################
## BEST ##
########################################################################################################################

# Original JSON file name and backup copy
original_json_filename = 'model_0.json'
backup_json_filename = 'model_best.json'

# Read content from the original JSON file
with open(original_json_filename, 'r') as original_file:
    data = json.load(original_file)

# Copy data to a new dictionary
backup_data = data.copy()

# Check if there are at least "split_point" entries in 'results'
if len(backup_data['results']) > split_point:
    # Remove the first "split_point" entries from 'results'
    backup_data['results'] = backup_data['results'][split_point:]

    # Overwrite the JSON file with updated data
    with open(backup_json_filename, 'w') as backup_file:
        json.dump(backup_data, backup_file, indent=4)
        print(f"First {split_point} entries successfully removed from 'results'.")

    ########################################################################################################################
    ## OVERWRITING THE REMAINING ENTRIES WITH THOSE PREDICTED BY THE ENSEMBLE ##

    # Load data from the CSV file
    csv_filename = 'predictions_best.csv'  # Replace with the actual name of your CSV file

    csv_data = []
    with open(csv_filename, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)

        # Skip the first row (column headers)
        next(reader)  # This step skips the first row

        for row in reader:
            csv_data.append([string_to_list(item) for item in row])

    with open(backup_json_filename, 'r') as jsonfile:
        backup_data = json.load(jsonfile)

    # Check if there are enough rows in the CSV to update all 'predicted_times' sections
    num_csv_rows = len(csv_data)
    num_results = len(backup_data['results'])

    if num_csv_rows < num_results:
        print(
            f"Warning: The number of rows in the CSV ({num_csv_rows}) is less than the number of 'results' sections to update ({num_results}).")
        print("Operation aborted.")
    else:
        # Sequentially update the 'predicted_times' sections of 'results'
        for i in range(num_results):
            backup_data['results'][i]['predicted_times'] = csv_data[i]

        # Overwrite the JSON file with updated data
        with open(backup_json_filename, 'w') as jsonfile:
            json.dump(backup_data, jsonfile, indent=4)
            print(
                f"Update complete. The first {num_results} 'predicted_times' sections have been successfully updated.")
else:
    print("Not enough items to remove.")
